In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score

In [34]:
data=pd.read_csv('D:\Subscribers (1).csv')

In [35]:
data.shape

(300000, 51)

In [36]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V42,V43,V44,V45,V46,V47,V48,V49,V50,Subscribers
0,-0.211875,743952.92,743952.92,200000.0,743952.92,200000.000,200000.0,-0.069396,943952.92,0.00,...,1.034232e+06,1966.57,1990.82,1752.89,100000.0,147.0,1826.046667,28.0,2557.22,0
1,0.241488,11295310.87,8034290.99,32000.0,8034290.99,18000.000,14000.0,0.167411,7508345.76,1160925.13,...,5.665658e+07,49957.65,11104.54,17711.99,38000.0,19.0,26467.683330,19.0,12807.07,0
2,0.459032,20363.68,317922.94,1650000.0,317922.94,1650000.000,1650000.0,46.068404,1655168.31,0.00,...,2.654720e+04,453.96,9.64,58.83,45072.0,0.0,315.506667,0.0,32.12,1
3,0.000000,856120.91,856120.91,0.0,856120.91,45554.885,0.0,0.000000,856120.91,0.00,...,8.506973e+05,2045.09,2045.09,2011.19,45072.0,30.0,2045.090000,19.0,2029.79,0
4,8.270783,194485.64,412330.33,835000.0,412330.33,835000.000,835000.0,-0.272225,2241.64,78625.00,...,5.572559e+04,70.01,100.60,89.59,31080.0,0.0,294.580000,0.0,26.95,0


In [37]:
y=pd.get_dummies(data['Subscribers'])

In [38]:
y

,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
299995,1,0
299996,1,0
299997,1,0
299998,1,0


In [39]:
x=data.drop(['Subscribers'],axis=1)

In [40]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,
                                               random_state=2)

In [10]:
# x_train

In [41]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [42]:
st=StandardScaler()

In [43]:
st.fit(x_train)

StandardScaler()

In [44]:
x_train=pd.DataFrame(st.transform(x_train))
x_test=pd.DataFrame(st.transform(x_test))

In [45]:
x_train.shape

(240000, 50)

# simple logistic regression

In [46]:
weights = tf.Variable(tf.random.normal(shape=(50, 2), dtype=tf.float64))
biases  = tf.Variable(tf.random.normal(shape=(2,), dtype=tf.float64))

In [48]:
def logistic_regression(x):
    lr = tf.add(tf.matmul(x, weights), biases)#wx+b
    return lr


def cross_entropy(y_true, y_pred):#softmax for cat var coz we converted to cat else binary
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return tf.reduce_mean(loss)

def accuracy(y_true, y_pred):
    y_true = tf.cast(tf.argmax(y_true, axis=1), dtype=tf.int32)
    preds = tf.cast(tf.argmax(y_pred, axis=1), dtype=tf.int32)
    preds = tf.equal(y_true, preds)
    return tf.reduce_mean(tf.cast(preds, dtype=tf.float32))

def roc_auc(y_true,x):
    y_true = tf.cast(tf.argmax(y_true, axis=1), dtype=tf.int32).numpy()
    y_pred=tf.nn.softmax(logistic_regression(x))[:,1]
    return(roc_auc_score(y_true,y_pred))

def grad(x, y):#to cal loss
    with tf.GradientTape() as tape:
        y_pred = logistic_regression(x)
        loss_val = cross_entropy(y, y_pred)
    return tape.gradient(loss_val, [weights, biases])#return 2 var

In [49]:
epochs=2000 #iterartion in alg to see data

In [50]:
y_true=tf.cast(tf.argmax(y_test.values,axis=1),dtype=tf.int32)


In [51]:
preds=tf.cast(tf.argmax(logistic_regression(x_test.values),axis=1),dtype=tf.int32)#cont 0 and 1

In [52]:
tf.reduce_mean(tf.cast(tf.equal(y_true,preds),dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=0.6953833>

In [53]:
learning_rate=0.01
for epoch in range(epochs):
    rand_ind=np.random.choice(range(x_train.shape[0]),100)# selection of batch,100 random sample
    outputs=y_train.iloc[rand_ind,:].values
    inputs=x_train.iloc[rand_ind,:].values
    
    dw,db=grad(inputs,tf.cast(outputs,'float32'))#cal grad dw=dl/dw
    
    weights.assign_sub(learning_rate*dw)
    biases.assign_sub(learning_rate*db)
    
    if epoch%50==0:#print the val
        print('epouch={}'.format(epoch),accuracy(y_test.values,logistic_regression(x_test.values)).numpy(),roc_auc(y_test.values,x_test.values))
    
    

epouch=0 0.69565 0.5116819354637383
epouch=50 0.70915 0.519238756990498
epouch=100 0.72255 0.5283581425154086
epouch=150 0.73441666 0.5371032972363021
epouch=200 0.7462 0.5450988317640568
epouch=250 0.75778335 0.5529644445509727
epouch=300 0.7683167 0.5593542260366638
epouch=350 0.7781 0.5654668653686361
epouch=400 0.78818333 0.5726825109347184
epouch=450 0.7968 0.5784915706857383
epouch=500 0.8048 0.5850497211584579
epouch=550 0.8096833 0.5893213365212698
epouch=600 0.81455 0.594437167718085
epouch=650 0.8179 0.599297559228883
epouch=700 0.82011664 0.6053741612889105
epouch=750 0.82145 0.6084325270396375
epouch=800 0.8224667 0.6127484577942764
epouch=850 0.82283336 0.6157427868073911
epouch=900 0.8234 0.6190446468995611
epouch=950 0.8236833 0.6230005683358152
epouch=1000 0.8237 0.6263587966216893
epouch=1050 0.82375 0.6293201694568067
epouch=1100 0.82376665 0.6328747671722611
epouch=1150 0.8239833 0.6354280894594005
epouch=1200 0.82441664 0.6362800676182309
epouch=1250 0.8243167 0.639

# classfier with a 1hidden layer

In [57]:
#hidden layer weights(50,20)
weights_h = tf.Variable(tf.random.truncated_normal(shape=(50, 20), dtype=tf.float64))
biases_h  = tf.Variable(tf.random.truncated_normal(shape=(20,), dtype=tf.float64))
#ooupt layer
weights_o = tf.Variable(tf.random.truncated_normal(shape=(20, 2), dtype=tf.float64))
biases_o  = tf.Variable(tf.random.truncated_normal(shape=(2,), dtype=tf.float64))

In [55]:
def logistic_regression(x):
    h_o = tf.add(tf.matmul(x, weights_h), biases_h)#ho-output of hidden layer
    h_o=tf.nn.relu(h_o)#activation func relu
    lr=tf.add(tf.matmul(h_o, weights_o), biases_o)#ho is i/p wrt weights to o/p layer
    return lr


def cross_entropy(y_true, y_pred):

    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return tf.reduce_mean(loss)


def accuracy(y_true, y_pred):
    y_true = tf.cast(tf.argmax(y_true, axis=1), dtype=tf.int32)
    preds = tf.cast(tf.argmax(y_pred, axis=1), dtype=tf.int32)
    preds = tf.equal(y_true, preds)
    return tf.reduce_mean(tf.cast(preds, dtype=tf.float32))

def roc_auc(y_true,x):
    y_true = tf.cast(tf.argmax(y_true, axis=1), dtype=tf.int32).numpy()
    y_pred=tf.nn.softmax(logistic_regression(x))[:,1]
    return(roc_auc_score(y_true,y_pred))

def grad(x, y):
    with tf.GradientTape() as tape:
        y_pred = logistic_regression(x)
        loss_val = cross_entropy(y, y_pred)
    return tape.gradient(loss_val, [weights_h, biases_h,weights_o, biases_o])

In [48]:
epochs=2000

In [58]:
learning_rate=0.01
for epoch in range(epochs):
    rand_ind=np.random.choice(range(x_train.shape[0]),100)
    outputs=y_train.iloc[rand_ind,:].values
    inputs=x_train.iloc[rand_ind,:].values
    
    dw_h,db_h,dw_o,db_o=grad(inputs,tf.cast(outputs,'float32')) #dh/dw,dh/db
    
    weights_h.assign_sub(learning_rate*dw_h)#updateding the weights
    biases_h.assign_sub(learning_rate*db_h)
    weights_o.assign_sub(learning_rate*dw_o)
    biases_o.assign_sub(learning_rate*db_o)
    
    if epoch%50==0:
        print(accuracy(y_test.values,logistic_regression(x_test.values)).numpy(),roc_auc(y_test.values,x_test.values))
    
    

0.24645 0.5380973202735381
0.7632 0.5750490050239142
0.7894167 0.5832840970482777
0.79466665 0.5874335920704606
0.7965 0.5956745141419634
0.7936 0.5975887517024265
0.79768336 0.6012882897821649
0.78891665 0.6054790435243419
0.79975 0.607536433750318
0.80836666 0.6113583881667111
0.80785 0.6133241612728897
0.8085 0.6124675761264071
0.8124667 0.6141138903928788
0.81015 0.6151812256867595
0.81591666 0.6180763296030123
0.8150333 0.6127972238722238
0.81905 0.6113101173692429
0.8197 0.6133386308872036
0.8189333 0.6189028247508385
0.82128334 0.6217868595879801
0.82028335 0.6244782566946154
0.82198334 0.624085629200296
0.82123333 0.6262820408469202
0.8237333 0.630263357374567
0.82425 0.6316249801233295
0.82406664 0.6329860519091924
0.8228 0.6329480882208672
0.8249 0.633974153072139
0.82411665 0.6388804737880807
0.82568336 0.6384328652761526
0.8255 0.6340130213555798
0.82625 0.6352355122957375
0.8257833 0.6370337770501977
0.82625 0.6396322255433606
0.82633334 0.6405099270268004
0.82448334 0.640

# classifier with 2 hidden layers

weights_h1 = tf.Variable(tf.random.truncated_normal(shape=(50, 30), dtype=tf.float64))
biases_h1  = tf.Variable(tf.random.truncated_normal(shape=(30,), dtype=tf.float64))

weights_h2 = tf.Variable(tf.random.truncated_normal(shape=(30, 10), dtype=tf.float64))
biases_h2  = tf.Variable(tf.random.truncated_normal(shape=(10,), dtype=tf.float64))

weights_o = tf.Variable(tf.random.truncated_normal(shape=(10, 2), dtype=tf.float64))
biases_o  = tf.Variable(tf.random.truncated_normal(shape=(2,), dtype=tf.float64))

In [72]:
epochs=2000

In [73]:
learning_rate=0.01
for epoch in range(epochs):
    rand_ind=np.random.choice(range(x_train.shape[0]),100)
    outputs=y_train.iloc[rand_ind,:].values
    inputs=x_train.iloc[rand_ind,:].values
    
    dw_h1,db_h1,dw_h2,db_h2,dw_o,db_o=grad(inputs,tf.cast(outputs,'float32'))
    
    weights_h1.assign_sub(learning_rate*dw_h1)
    biases_h1.assign_sub(learning_rate*db_h1)
    weights_h2.assign_sub(learning_rate*dw_h2)
    biases_h2.assign_sub(learning_rate*db_h2)
    weights_o.assign_sub(learning_rate*dw_o)
    biases_o.assign_sub(learning_rate*db_o)
    
    if epoch%50==0:
        print(accuracy(y_test.values,logistic_regression(x_test.values)).numpy(),roc_auc(y_test.values,x_test.values))
    

0.6103 0.5712062556655887
0.73765 0.572267332060002
0.65056664 0.5749276144160623
0.68745 0.5704915630035181
0.7111833 0.5652227692662954
0.7266 0.5764507171618355
0.73866665 0.5698978262356639
0.7375 0.5655830833726673
0.73035 0.571623401008953
0.76705 0.5716266257049326
0.7435167 0.5895714805160098
0.7861 0.5664943310723873
0.77743334 0.587868504013577
0.74803334 0.602347744548185
0.7996167 0.5897901864113808
0.8010167 0.5807453606247166
0.81525 0.5816843889679596
0.8168 0.5951303298441387
0.81736666 0.6028942001026447
0.81703335 0.6063052153233739
0.82233334 0.5842064851396827
0.8254 0.5906774457334952
0.8258833 0.5920278460322689
0.82545 0.6101107082578062
0.8258167 0.6120629505357466
0.8258 0.6143219111247764
0.8265833 0.5998559210525338
0.8265667 0.6212295430308238
0.8267 0.6133274377437519
0.8268333 0.6138695295548182
0.82701665 0.6052127301361288
0.8171833 0.6209233834977108
0.82698333 0.6190628677506402
0.8234 0.6287484633898712
0.82755 0.6251121053199735
0.8271833 0.625853351